# Basics

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size=8
batch_size=4
max_iters=10000
eval_interval=2500
learning_rate=3e-4
eval_iters=500
dropout=0.2

cuda


In [2]:
with open('Wizard_of_OZ.txt','r',encoding='utf-8') as f:
    text=f.read()
chars=sorted(set(text))
print(chars)
vocabulary_size=len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}

encode=lambda s:[string_to_int[c] for c in s]
decode=lambda l:''.join(int_to_string[i] for i in l)

data=torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

# decoded_hello=decode()
# print(decoded_hello)

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
print(encode('sahil'))

[72, 54, 61, 62, 65]


In [5]:
decode([72, 54, 61, 62, 65])

'sahil'

In [6]:
print(encode(text[:100]))

[80, 1, 1, 28, 39, 42, 39, 44, 32, 49, 1, 25, 38, 28, 1, 44, 32, 29, 1, 47, 33, 50, 25, 42, 28, 1, 33, 38, 1, 39, 50, 0, 0, 1, 1, 26, 49, 0, 0, 1, 1, 36, 11, 1, 30, 42, 25, 38, 35, 1, 26, 25, 45, 37, 0, 0, 1, 1, 25, 45, 44, 32, 39, 42, 1, 39, 30, 1, 44, 32, 29, 1, 47, 33, 50, 25, 42, 28, 1, 39, 30, 1, 39, 50, 9, 1, 44, 32, 29, 1, 36, 25, 38, 28, 1, 39, 30, 1, 39, 50]


In [7]:
text[:100]

'\ufeff  DOROTHY AND THE WIZARD IN OZ\n\n  BY\n\n  L. FRANK BAUM\n\n  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ'

In [8]:
vocabulary_size

81

# Training And Validation Split

In [9]:
n=int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    # print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x,y

x,y=get_batch('train')
print('inputs')
print(x)
print('targets')
print(y)

inputs
tensor([[ 1, 73, 68,  1, 54, 65, 65,  1],
        [58,  1, 66, 68, 74, 73, 61,  1],
        [66,  1, 61, 68, 76,  1, 73, 68],
        [78, 58, 72,  1, 76, 58, 71, 58]], device='cuda:0')
targets
tensor([[73, 68,  1, 54, 65, 65,  1, 73],
        [ 1, 66, 68, 74, 73, 61,  1, 68],
        [ 1, 61, 68, 76,  1, 73, 68,  1],
        [58, 72,  1, 76, 58, 71, 58,  1]], device='cuda:0')


In [10]:
x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print('when input is ',context,' target is ',target)

when input is  tensor([80])  target is  tensor(1)
when input is  tensor([80,  1])  target is  tensor(1)
when input is  tensor([80,  1,  1])  target is  tensor(28)
when input is  tensor([80,  1,  1, 28])  target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39])  target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42])  target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39])  target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44])  target is  tensor(32)


In [22]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out    

In [19]:
device

'cuda'

In [18]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

In [20]:
model = BigramLanguageModel(vocabulary_size)
m = model.to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


xqoDYfr39Xvyn:EzuaCmjQ3UKuIVtE(DwykRNB*.wXvoF47F)e84nqAquuJVJ'l))4axP﻿UGfi6fr1O_mTM7uB2ycvKOw(:VW"q"u&iuyUFbuQ-L&LS7h(z'6cV1!:KyPcC?AW"g&vH﻿3T 6!)ED6T.WRB_RA"4j]mfQKXjUbkti?p﻿9nR55M7jrKWt8]wy,b)7_R]7Agk)zS[R?DUmu
xWfO0teF:,FJ-MONtJ yb9Yo*EjSy4t1fP﻿GA;iiqz!Y4g:'sTWZAg'oJeAsr_,b'1z)EwX:AgU(1vFSJ3'CM]LhEt1C,uvk"0Aa-cynP﻿R*HucCdR*X!)!sf?HQ9;ituWtS8L
E47Rt,AW72z aZz-LU1C3uN,XJer:uxYoSjQamOfZ:
sWa;08DwjXWbn9eJeYlJfk_QP_h8k?e8tyTvgBlumzr:*g5cVAgmm[0OjXQp1z4YfH5Wt8
e1[EsbuttYftRt _)?_Hu*QelnSUgCw'9CV-pe


In [28]:
optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
    if iter%eval_iters==0:
        losses=estimate_loss()
        print(f"step : {iter},train losses : {losses['train']:.4f}, validation losses: {losses['val']:.4f}")
    xb,yb=get_batch('train')

    logits,loss=model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print( loss.item())

step : 0,train losses : 2.633108377456665, validation losses: 2.6702687740325928
step : 500,train losses : 2.645479440689087, validation losses: 2.6736841201782227
step : 1000,train losses : 2.6114025115966797, validation losses: 2.651033878326416
step : 1500,train losses : 2.5916192531585693, validation losses: 2.6586270332336426
step : 2000,train losses : 2.613177537918091, validation losses: 2.6312878131866455
step : 2500,train losses : 2.5859017372131348, validation losses: 2.62654447555542
step : 3000,train losses : 2.590078353881836, validation losses: 2.611853837966919
step : 3500,train losses : 2.5787744522094727, validation losses: 2.5963566303253174
step : 4000,train losses : 2.587779998779297, validation losses: 2.603783130645752
step : 4500,train losses : 2.561534881591797, validation losses: 2.604968309402466
step : 5000,train losses : 2.5418245792388916, validation losses: 2.60144305229187
step : 5500,train losses : 2.5533199310302734, validation losses: 2.582801103591919

In [29]:
context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


qAime w?﻿Bke ad the sthev8]ndin. s, k_; awd fr tralt. tror isit tlithk."
"FULUS8﻿6
" an tss?4Qce arosasere g tan,BBld eskes t thqshe wighevorvP﻿ ses Is sth, anare llippseang:s," firl.s wiptinse, tsroun maugind d avfind d, d oe qI'med bus. thastine ru, brr Va cly t?lon tomealerofie thinn.
SB0trureri*ag, ir, oved b.
"'vesthe,"Vh."&)S."
(Yfanto be red semus thed

wond fa, d ymeatholese an zay'sence. d SEil t m'th ar ilerain kead banr sa, THICI'laca  ghed a ared apVx'ormFthithery b, wevinoon,"I t
ke
